In [1]:
import sqlite3
import pandas as pd
import os
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
# import musicbrainzngs

os.chdir('../')
os.chdir('../')

from plagdet.src.defaults import *

In [3]:
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


### load model

In [4]:
mel_2bar_config = configs.CONFIG_MAP[MUSICVAE_2BAR_BIG_CONFIG]
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path=MUSICVAE_2BAR_BIG_PATH)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\contrib\rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\contrib\rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.
INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(


### extract melodies

In [42]:
input_mel_midi_data = [os.path.join(MIDI_DATA_PATH, 'aa_midis', file) for file in os.listdir(os.path.join(MIDI_DATA_PATH, 'aa_midis'))]
input_mel_midi_data = [input_mel_midi_data[0]]
# mel_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_mel_midi_data]
print(input_mel_midi_data)
mel_input_seqs = []
for midi_path in input_mel_midi_data:
    with open(midi_path, 'rb') as midi_file:
        midi_data = midi_file.read()
        mel_input_seqs.append(mm.midi_to_sequence_proto(midi_data))
  
extracted_mels = []
for ns in mel_input_seqs:
  extracted_mels.extend(
      mel_2bar_config.data_converter.from_tensors(
          mel_2bar_config.data_converter.to_tensors(ns)[1]))
for i, ns in enumerate(extracted_mels):
  print("Melody", i)

['plagdet/data/midi_databases\\aa_midis\\Heartbreaker.mid']
Melody 0
Melody 1
Melody 2
Melody 3
Melody 4


#### looking at extracted melodies

In [ ]:
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced

In [41]:
len(extracted_mels)

5

### encode notesequences

In [38]:
note_sequences = extracted_mels.copy()
inputs = []
controls = []
lengths = []
note_sequence = note_sequences[0]
extracted_tensors = mel_2bar._config.data_converter.to_tensors(note_sequence)
if not extracted_tensors.inputs:
    raise Exception('No examples extracted from NoteSequence: %s' % note_sequence)
# if len(extracted_tensors.inputs) > 1:
#     raise Exception('Multiple (%d) examples extracted from NoteSequence: %s' % (len(extracted_tensors.inputs), note_sequence))

inputs.append(extracted_tensors.inputs[0])
controls.append(extracted_tensors.controls[0])
lengths.append(extracted_tensors.lengths[0])

vec = mel_2bar.encode_tensors(inputs, lengths, controls)[0]

In [ ]:
note_sequences = mel_input_seqs.copy()
len

inputs = []
controls = []
lengths = []

vecs = []

for note_sequence in note_sequences:
    extracted_tensors = mel_2bar._config.data_converter.to_tensors(note_sequence)
    if not extracted_tensors.inputs:
        raise Exception('No examples extracted from NoteSequence: %s' % note_sequence)
    for i in range(len(extracted_tensors.inputs)):
        inputs.append(extracted_tensors.inputs[i])
        controls.append(extracted_tensors.controls[i])
        lengths.append(extracted_tensors.lengths[i])
return self.encode_tensors(inputs, lengths, controls)

In [10]:
encoded_mels = mel_2bar.encode(mel_input_seqs)
print(encoded_mels)

MultipleExtractedExamplesError: Multiple (5) examples extracted from NoteSequence: ticks_per_quarter: 120
time_signatures {
  numerator: 4
  denominator: 4
}
time_signatures {
  time: 47.5
  numerator: 3
  denominator: 4
}
time_signatures {
  time: 49.375
  numerator: 2
  denominator: 4
}
time_signatures {
  time: 50.625
  numerator: 4
  denominator: 4
}
time_signatures {
  time: 90.625
  numerator: 3
  denominator: 4
}
time_signatures {
  time: 92.5
  numerator: 2
  denominator: 4
}
time_signatures {
  time: 93.75
  numerator: 4
  denominator: 4
}
time_signatures {
  time: 149.93821641666668
  numerator: 5
  denominator: 4
}
time_signatures {
  time: 152.86485198333335
  numerator: 4
  denominator: 4
}
time_signatures {
  time: 167.81696194166668
  numerator: 5
  denominator: 4
}
time_signatures {
  time: 170.73287599166667
  numerator: 4
  denominator: 4
}
time_signatures {
  time: 173.06297199166667
  numerator: 5
  denominator: 4
}
time_signatures {
  time: 175.97559199166668
  num